In [ ]:
# 1. Configuração do ambiente
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Evitar quebra de linha no print do DataFrame
pd.set_option('display.width', 0)  # ou use um valor grande como 2000
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)  # Impede quebra em várias linhas

# 2. Leitura do CSV (ajuste o caminho se necessário)
df = pd.read_csv("BRA.csv")

# Country: País onde ocorreu a partida (ex: Brazil)
# League: Nome da liga (ex: Serie A)
# Season: Ano da temporada (ex: 2012)
# Date: Data do jogo (formato dia/mês/ano)
# Time: Horário da partida (horário local, pode estar ausente em alguns jogos)
# Home: Nome do time mandante
# Away: Nome do time visitante
# HG: Gols marcados pelo time da casa (Home Goals)
# AG: Gols marcados pelo time visitante (Away Goals)
# Res: Resultado da partida (H = vitória do mandante, D = empate, A = vitória do visitante)

# PSCH, PSCD, PSCA: Odds de fechamento (closing odds) do mercado de apostas da Pinnacle para vitória do mandante (H), empate (D) e vitória do visitante (A)
# MaxCH, MaxCD, MaxCA: Maiores odds disponíveis entre as casas de apostas para H, D e A
# AvgCH, AvgCD, AvgCA: Odds médias do mercado para H, D e A

# BFECH, BFECD, BFECA: Odds do site Betfair Exchange para H, D e A (podem estar ausentes)

# 3. Visualização inicial
print(df.head())
print(" ")
print(df.info())
print(" ")
print(df.describe())

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

In [ ]:
df['Home'] = df['Home'].str.strip().str.title()
df['Away'] = df['Away'].str.strip().str.title()

In [ ]:
df.drop(columns=["BFECH", "BFECD", "BFECA"], inplace=True)

In [ ]:
print(df)

In [ ]:
# Padronização de Nomes de Times (Verificação)
print("\nNomes únicos de times da casa:")
print(df['Home'].unique())
print("\nNomes únicos de times visitantes:")
print(df['Away'].unique())

In [ ]:
missing = df.isnull().sum()
print(missing[missing > 0])

df_missing = df[df.isna().any(axis=1)]
print(df_missing)

df.dropna(subset=['HG', 'AG', 'AvgCH', 'AvgCD', 'AvgCA'], inplace=True)

In [ ]:
# 7. Colunas auxiliares
df["GoalDiff"] = df["HG"] - df["AG"]
df['Total_Goals'] = df['HG'] + df['AG']
df["is_home_win"] = df["Res"] == "H"
df["is_draw"] = df["Res"] == "D"
df["is_away_win"] = df["Res"] == "A"

In [ ]:
df["ProbH"] = 1 / df["AvgCH"]
df["ProbD"] = 1 / df["AvgCD"]
df["ProbA"] = 1 / df["AvgCA"]

# Normalização das probabilidades
df["sum_probs"] = df["ProbH"] + df["ProbD"] + df["ProbA"]
df["ProbH"] /= df["sum_probs"]
df["ProbD"] /= df["sum_probs"]
df["ProbA"] /= df["sum_probs"]

df['Margin'] = df["sum_probs"] - 1

In [ ]:
print(df)

In [ ]:
df['Res'].value_counts().plot(kind='bar', color='blue', alpha=0.4)
plt.title('Frequência de Resultados')
plt.xticks(ticks=[0, 1, 2], labels=['Vitória Casa', 'Empate', 'Vitória Visitante'], rotation=0)
plt.show()

In [ ]:
# Criar nova coluna categorizando o resultado
df["Casa_Venceu"] = df["Res"].apply(lambda x: "Vitória Casa" if x == "H" else "Empate ou Derrota")

# Contar e plotar
df["Casa_Venceu"].value_counts().plot(kind="bar", color="blue", alpha=0.4)
plt.title("Frequência: Vitória em Casa vs. Empate ou Derrota")
plt.xticks(rotation=0)
plt.ylabel("Número de Partidas")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(df['HG'], bins=range(0, 7), kde=False)
plt.title("Distribuição de Gols Marcados (Casa)")
plt.show()

sns.histplot(df['AG'], bins=range(0, 7), kde=False)
plt.title("Distribuição de Gols Marcados (Fora)")
plt.show()


In [ ]:
sns.boxplot(x='Season', y='HG', data=df)
plt.xticks(rotation=45)
plt.title("Total de Gols por Jogo (mandante) ao Longo das Temporadas")
plt.show()

In [ ]:
sns.boxplot(x='Season', y='AG', data=df)
plt.xticks(rotation=45)
plt.title("Total de Gols por Jogo (visitante) ao Longo das Temporadas")
plt.show()

In [ ]:
sns.boxplot(x='Season', y='Total_Goals', data=df)
plt.xticks(rotation=45)
plt.title("Total de Gols por Jogo ao Longo das Temporadas")
plt.show()

In [ ]:
corr = df[['HG', 'AG', 'ProbH', 'ProbD', 'ProbA']].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlação entre Gols e Probabilidades")
plt.show()

In [ ]:
df['Year'] = df['Date'].dt.year
sns.lineplot(data=df.groupby('Year')['Margin'].mean().reset_index(), x='Year', y='Margin')
plt.title("Evolução da Margem das Casas de Apostas")
plt.show()

In [ ]:
# Calcular margens implícitas para cada conjunto de odds
for prefix in ["PS", "Max", "Avg"]:
    df[f"{prefix}_H_Prob"] = 1 / df[f"{prefix}CH"]
    df[f"{prefix}_D_Prob"] = 1 / df[f"{prefix}CD"]
    df[f"{prefix}_A_Prob"] = 1 / df[f"{prefix}CA"]
    df[f"{prefix}_Sum"] = (
        df[f"{prefix}_H_Prob"] + df[f"{prefix}_D_Prob"] + df[f"{prefix}_A_Prob"]
    )
    df[f"{prefix}_Margin"] = df[f"{prefix}_Sum"] - 1

# Plotar as distribuições de margem
plt.figure(figsize=(10, 6))
for prefix in ["PS", "Max", "Avg"]:
    sns.kdeplot(
        df[f"{prefix}_Margin"], label=prefix, fill=True, alpha=0.4
    )
plt.title("Distribuição das Margens Implícitas das Casas de Apostas")
plt.xlabel("Margem")
plt.ylabel("Densidade")
plt.legend(title="Fonte da Odd")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Escolha um jogo com odds realistas (por exemplo, a 1ª linha)
linha = df.iloc[5039]

# PSCH  PSCD   PSCA

odd_home = linha["PSCH"]
odd_draw = linha["PSCD"]
odd_away = linha["PSCA"]

# Odds com overround (o que o público realmente recebe)
odds_com_spread = {"H": odd_home, "D": odd_draw, "A": odd_away}

# Cálculo das probabilidades implícitas
prob_H = 1 / odd_home
prob_D = 1 / odd_draw
prob_A = 1 / odd_away
soma_probs = prob_H + prob_D + prob_A

# Probabilidades normalizadas (como o público aposta)
probabilidades_normalizadas = {
    "H": prob_H / soma_probs,
    "D": prob_D / soma_probs,
    "A": prob_A / soma_probs
}

# Odds justas (sem overround)
odds_justas = {k: 1 / v for k, v in probabilidades_normalizadas.items()}

# Simulação
np.random.seed(42)
n_apostas = 1_000_000
stake = 1.0
apostas = np.random.choice(["H", "D", "A"], size=n_apostas, p=list(probabilidades_normalizadas.values()))

# Resultado real (pegando da coluna Res)
resultado_real = linha["Res"]

# Lucro da casa com odds COM spread
pagamento_com_spread = np.sum([
    stake * odds_com_spread[res] if res == resultado_real else 0
    for res in apostas
])
lucro_casa_com_spread = n_apostas * stake - pagamento_com_spread

# Lucro da casa com odds JUSTAS (para comparação)
pagamento_justo = np.sum([
    stake * odds_justas[res] if res == resultado_real else 0
    for res in apostas
])
lucro_casa_justa = n_apostas * stake - pagamento_justo

# Formatando os dicionários para remoção de np.float64
odds_com_spread_fmt = {k: round(float(v), 3) for k, v in odds_com_spread.items()}
odds_justas_fmt = {k: round(float(v), 3) for k, v in odds_justas.items()}
probs_fmt = {k: round(float(v), 4) for k, v in probabilidades_normalizadas.items()}

# Exibir resultados limpos
print(f"🎯 Jogo: {linha['Home']} x {linha['Away']} — Resultado real: {resultado_real}")
print(f"\nOdds com spread: {odds_com_spread_fmt}")
print(f"Odds justas (normalizadas): {odds_justas_fmt}")
print(f"\n🔢 Probabilidades implícitas normalizadas: {probs_fmt}")
print(f"\n💰 Lucro da casa com spread: R${lucro_casa_com_spread:,.2f}")
print(f"💸 Lucro da casa com odds justas: R${lucro_casa_justa:,.2f}")
print(f"\n📈 Diferença: R${(lucro_casa_com_spread - lucro_casa_justa):,.2f}")


In [ ]:
# Diferença entre maior e menor odd para cada linha (Avg odds) # PSCH  PSCD   PSCA
df["OddsSpread"] = df[["PSCH","PSCD","PSCA"]].max(axis=1) - df[["PSCH","PSCD","PSCA"]].min(axis=1)

# Correlação entre spread das odds e a margem da casa
correlation = df[["OddsSpread", "Avg_Margin"]].corr()
print(correlation)

# Visualização com linha de tendência linear
plt.figure(figsize=(8, 5))
sns.regplot(
    data=df,
    x="OddsSpread",
    y="Avg_Margin",
    scatter_kws={"alpha": 0.3},
    line_kws={"color": "red"}
)
plt.title("Correlação entre Dispersão das Odds e Margem da Casa")
plt.xlabel("Diferença Máxima entre Odds (Spread)")
plt.ylabel("Margem da Casa (Avg)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Escolha um jogo com odds realistas (por exemplo, a 1ª linha)
linha = df.iloc[5039]

# PSCH  PSCD   PSCA

odd_home = linha["PSCH"]
odd_draw = linha["PSCD"]
odd_away = linha["PSCA"]

print(f"🎯 Jogo: {linha['Home']} x {linha['Away']} — Resultado real: {resultado_real}")

# Aposta unitária e número total de apostas
stake = 1.0
total_apostas = 1_000_000

# Probabilidades implícitas normalizadas
inv_probs = {
    "H": 1 / odd_home,
    "D": 1 / odd_draw,
    "A": 1 / odd_away
}
soma = sum(inv_probs.values())
prob_imp = {k: v / soma for k, v in inv_probs.items()}

# Mostrar as probabilidades implícitas
print("🎯 Probabilidades Implícitas Normalizadas:")
for k, v in prob_imp.items():
    print(f"  {k}: {v:.4f}")

# Criar cenários variando a preferência do público
cenarios = {
    "proporcional": [prob_imp["H"], prob_imp["D"], prob_imp["A"]],
    "enviesado_home": [prob_imp["H"] * 1.3, prob_imp["D"] * 0.85, prob_imp["A"] * 0.85],
    "enviesado_draw": [prob_imp["H"] * 0.85, prob_imp["D"] * 1.3, prob_imp["A"] * 0.85],
    "enviesado_away": [prob_imp["H"] * 0.85, prob_imp["D"] * 0.85, prob_imp["A"] * 1.3],
}

# Normalizar cada cenário
for k in cenarios:
    total = sum(cenarios[k])
    cenarios[k] = [x / total for x in cenarios[k]]

# Odds por resultado
odds = {"H": odd_home, "D": odd_draw, "A": odd_away}
resultados = ["H", "D", "A"]

# Simulação
def simular_lucro(distribuicao, resultado_real, odds):
    apostas = np.random.choice(["H", "D", "A"], size=total_apostas, p=distribuicao)
    odds_apostas = np.vectorize(odds.get)(apostas)
    ganhos = np.where(apostas == resultado_real, odds_apostas * stake, 0)
    lucro_casa = total_apostas * stake - ganhos.sum()
    return lucro_casa

# Executar simulações
for nome, dist in cenarios.items():
    print(f"\n📊 Cenário: {nome.replace('_', ' ').title()}")
    for resultado in resultados:
        lucro = simular_lucro(dist, resultado, odds)
        print(f"  Resultado real: {resultado} → Lucro da casa: R${lucro:,.2f}")


In [ ]:
# Criar faixas de odds balanceadas usando PSCH (Odds da Casa)
df['Odds_Faixa'], bins = pd.qcut(df['PSCH'], q=10, retbins=True, labels=[f'Faixa {i+1}' for i in range(10)])

# Marcar vitória em casa como 1, outras como 0
df['Vitoria_Casa'] = (df['Res'] == 'H').astype(int)

# Calcular probabilidade real por faixa (frequência de vitórias em casa)
real_prob = df.groupby('Odds_Faixa', observed=True)['Vitoria_Casa'].mean()

# Calcular média da probabilidade normalizada por faixa
norm_prob = df.groupby('Odds_Faixa', observed=True)['ProbH'].mean()

counts = df.groupby('Odds_Faixa', observed=True).size()

# Criar DataFrame comparativo
comparacao_prob = pd.DataFrame({
    'Prob_Real': real_prob,
    'Prob_Normalizada': norm_prob,
    'Num_Jogos': counts
}).reset_index()

total_partidas = counts.sum()
print(f"Total de partidas: {total_partidas}")
print(comparacao_prob)


# Criar rótulos com os ranges das odds
ranges = [f"{bins[i]:.2f} - {bins[i+1]:.2f}" for i in range(len(bins) - 1)]
comparacao_prob['Odds_Range'] = ranges

# Plotando gráfico de barras lado a lado
plt.figure(figsize=(12, 6))
x = range(len(comparacao_prob))

plt.bar(x, comparacao_prob['Prob_Real'], width=0.4, label='Probabilidade Real', alpha=0.7)
plt.bar([p + 0.4 for p in x], comparacao_prob['Prob_Normalizada'], width=0.4, label='Probabilidade Normalizada', color='blue', alpha=0.7)

# Ajustar eixo X com os ranges de odds
plt.xticks([p + 0.2 for p in x], comparacao_prob['Odds_Range'], rotation=45)
plt.xlabel('Faixas de Odds (PSCH)')
plt.ylabel('Probabilidade')
plt.title('Comparação: Probabilidade Real vs. Normalizada por Faixa de Odds da Casa')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Identificar faixas de value bet
faixas_valor = comparacao_prob[
    comparacao_prob["Prob_Real"] > comparacao_prob["Prob_Normalizada"]
]["Odds_Faixa"].tolist()

print(faixas_valor)

In [ ]:
# Simular apostas apenas nos jogos onde a PSCH cai nas faixas de valor
df["Aposta_Casa"] = df["Odds_Faixa"].isin(faixas_valor)
df["Lucro"] = 0.0
df.loc[df["Aposta_Casa"] & (df["Res"] == "H"), "Lucro"] = df["PSCH"] - 1
df.loc[df["Aposta_Casa"] & (df["Res"] != "H"), "Lucro"] = -1

# Avaliar desempenho
total_apostas = df["Aposta_Casa"].sum()
lucro_total = df["Lucro"].sum()
roi = lucro_total / total_apostas if total_apostas > 0 else 0

print(f"Apostas simuladas: {total_apostas}")
print(f"Lucro total: {lucro_total:.2f}")
print(f"ROI: {roi:.2%}")

In [ ]:
# Dados até 2024 para calibrar a estratégia
df_treino = df[df["Season"] <= 2024].dropna(subset=["PSCH", "ProbH", "Res"]).copy()

# Dados de 2025 para testar a estratégia
df_teste = df[df["Season"] == 2025].dropna(subset=["PSCH", "ProbH", "Res"]).copy()

In [ ]:
# Faixas de odds com PSCH (apenas no treino)
df_treino["Odds_Faixa"], bins = pd.qcut(
    df_treino["PSCH"], q=10, retbins=True, labels=[f"Faixa {i+1}" for i in range(10)]
)

# Marcar vitórias da casa
df_treino["Vitoria_Casa"] = (df_treino["Res"] == "H").astype(int)

# Probabilidades reais e normalizadas por faixa
real_prob = df_treino.groupby("Odds_Faixa", observed=True)["Vitoria_Casa"].mean()
norm_prob = df_treino.groupby("Odds_Faixa", observed=True)["ProbH"].mean()

counts = df_treino.groupby('Odds_Faixa', observed=True).size()

total_partidas = counts.sum()
print(f"Total de partidas: {total_partidas}")

# Criar DataFrame comparativo
comparacao_prob = pd.DataFrame({
    'Prob_Real': real_prob,
    'Prob_Normalizada': norm_prob,
    'Num_Jogos': counts
}).reset_index()

# Adicionar os ranges
ranges = [f"{bins[i]:.2f} - {bins[i+1]:.2f}" for i in range(len(bins) - 1)]
comparacao_prob["Odds_Range"] = ranges

print(comparacao_prob)

In [ ]:
# Identificar faixas de value bet
faixas_valor = comparacao_prob[
    comparacao_prob["Prob_Real"] > comparacao_prob["Prob_Normalizada"]
]["Odds_Faixa"].tolist()

print(faixas_valor)

In [ ]:
# Reusar os mesmos bins para rotular os dados de 2025
df_teste["Odds_Faixa"] = pd.cut(df_teste["PSCH"], bins=bins, labels=[f"Faixa {i+1}" for i in range(10)])

# Marcar onde apostar
df_teste["Aposta_Casa"] = df_teste["Odds_Faixa"].isin(faixas_valor)

# Calcular lucro
df_teste["Lucro"] = 0.0
df_teste.loc[df_teste["Aposta_Casa"] & (df_teste["Res"] == "H"), "Lucro"] = df_teste["PSCH"] - 1
df_teste.loc[df_teste["Aposta_Casa"] & (df_teste["Res"] != "H"), "Lucro"] = -1

# Avaliação
total_apostas = df_teste["Aposta_Casa"].sum()
lucro_total = df_teste["Lucro"].sum()
roi = lucro_total / total_apostas if total_apostas > 0 else 0

print(f"Apostas simuladas em 2025: {total_apostas}")
print(f"Lucro total: {lucro_total:.2f}")
print(f"ROI: {roi:.2%}")

In [ ]:
# Filtrar apenas as apostas realizadas
df_apostas = df_teste[df_teste["Aposta_Casa"]].copy()

# Agrupar por faixa
resultado_por_faixa = df_apostas.groupby("Odds_Faixa", observed=True).agg(
    Total_Apostas=("Aposta_Casa", "sum"),
    Lucro_Total=("Lucro", "sum"),
    Media_Odd=("PSCH", "mean")
)

# Calcular ROI por faixa
resultado_por_faixa["ROI"] = resultado_por_faixa["Lucro_Total"] / resultado_por_faixa["Total_Apostas"]

# Resetar índice para exibir como tabela
resultado_por_faixa = resultado_por_faixa.reset_index()

# Exibir
print(resultado_por_faixa)

In [ ]:
# Soma dos lucros e apostas por faixa
lucro_somado = resultado_por_faixa["Lucro_Total"].sum()
apostas_somadas = resultado_por_faixa["Total_Apostas"].sum()
roi_somado = lucro_somado / apostas_somadas if apostas_somadas > 0 else 0

# Comparar com o resultado total
print("=== Verificação de consistência ===")
print(f"Lucro total (somado por faixa): {lucro_somado:.2f}  |  Lucro total direto: {lucro_total:.2f}")
print(f"Apostas totais (somadas): {apostas_somadas}  |  Apostas totais direto: {total_apostas}")
print(f"ROI (calculado por faixa): {roi_somado:.2%}  |  ROI direto: {roi:.2%}")